# THE MUSIC MAKER WITH LSTM NEURAL NETWORKS. (Part 2)

Author: Matt Mills; MANTIS project. This is a follow up from the last example where we made a song from training a megaman midi. In that, we only tracked notes and chords, we look to expand that in this document to rests and try more MIDI musical inputs. Here, I will be a lot less descriptive on parts that were covered in part 1

# Table of Contents
1. [Import and Path Setups](#first-section)
2. [Song Load](#second-section)
3. [Making LSTM NN Model](#third-section)
4. [Predicting new music](#fourth-section)

## Import and Path Setups <a class = "anchor" id = "first-section"></a> 

In [9]:
from music21 import *
import numpy as np
import os
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint
import glob

print(os.getcwd())
print(sys.path)
e = environment.Environment()
e['musicxmlPath'] ='/usr/bin/musescore'
e['musescoreDirectPNGPath'] = '/usr/bin/musescore'

/home/millsms/Code/MattsMusicMaker
['/home/millsms/anaconda3/lib/python36.zip', '/home/millsms/anaconda3/lib/python3.6', '/home/millsms/anaconda3/lib/python3.6/lib-dynload', '', '/home/millsms/.local/lib/python3.6/site-packages', '/home/millsms/anaconda3/lib/python3.6/site-packages', '/home/millsms/.local/lib/python3.6/site-packages/IPython/extensions', '/home/millsms/.ipython']


## Song load <a class = "anchor" id = "second-section"></a> 

This is the first difference, let's include more songs. This is the second difference, let's try to account for more aspects than just notes and chords, now include rests

In [59]:
notesFull = []
for file in glob.glob("MegaManMidi_Multiple/*.mid"):
    myScore = converter.parse(file)
    myScoreParts = instrument.partitionByInstrument(myScore)
    if myScoreParts: #More than one instrument
        notes_to_parse = myScoreParts.parts[0].recurse() #takes first instrument
    else:
        notes_to_parse = myScoreParts.flat.notes
    
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notesFull.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notesFull.append('.'.join(str(n) for n in element.normalOrder))
        elif isinstance(element, note.Rest):
            notesFull.append(str(element.name) + str(element.duration.type))



In [11]:
print(len(notesFull))
notes = notesFull # Use all the notes
pitchList = sorted(set(item for item in notes))
print(pitchList)

3832
['0.3', '0.4', '0.5', '1.4', '1.5', '10.1', '10.2', '10.3', '11.2', '11.3', '11.4', '2.5', '2.6', '2.7', '3.7', '3.8', '4.7', '4.8', '4.9', '5.10', '5.8', '5.9', '6.10', '6.11', '6.9', '7.10', '7.11', '8.0', '8.1', '8.11', '9.1', '9.2', 'A2', 'A3', 'A4', 'A5', 'B-2', 'B-3', 'B-4', 'B-5', 'B4', 'B5', 'C#3', 'C#4', 'C#5', 'C4', 'C5', 'C6', 'D5', 'D6', 'E-3', 'E-4', 'E-5', 'E-6', 'F#2', 'F#3', 'F#4', 'F#5', 'F3', 'F4', 'F5', 'G#2', 'G#3', 'G#4', 'G#5', 'G2', 'G3', 'G4', 'G5', 'G6', 'rest16th', 'rest32nd', 'restbreve', 'restcomplex', 'resteighth', 'resthalf', 'restinexpressible', 'restquarter']


In [12]:
note2Int = dict((note, number) for number, note in enumerate(pitchList))
print(note2Int)

{'0.3': 0, '0.4': 1, '0.5': 2, '1.4': 3, '1.5': 4, '10.1': 5, '10.2': 6, '10.3': 7, '11.2': 8, '11.3': 9, '11.4': 10, '2.5': 11, '2.6': 12, '2.7': 13, '3.7': 14, '3.8': 15, '4.7': 16, '4.8': 17, '4.9': 18, '5.10': 19, '5.8': 20, '5.9': 21, '6.10': 22, '6.11': 23, '6.9': 24, '7.10': 25, '7.11': 26, '8.0': 27, '8.1': 28, '8.11': 29, '9.1': 30, '9.2': 31, 'A2': 32, 'A3': 33, 'A4': 34, 'A5': 35, 'B-2': 36, 'B-3': 37, 'B-4': 38, 'B-5': 39, 'B4': 40, 'B5': 41, 'C#3': 42, 'C#4': 43, 'C#5': 44, 'C4': 45, 'C5': 46, 'C6': 47, 'D5': 48, 'D6': 49, 'E-3': 50, 'E-4': 51, 'E-5': 52, 'E-6': 53, 'F#2': 54, 'F#3': 55, 'F#4': 56, 'F#5': 57, 'F3': 58, 'F4': 59, 'F5': 60, 'G#2': 61, 'G#3': 62, 'G#4': 63, 'G#5': 64, 'G2': 65, 'G3': 66, 'G4': 67, 'G5': 68, 'G6': 69, 'rest16th': 70, 'rest32nd': 71, 'restbreve': 72, 'restcomplex': 73, 'resteighth': 74, 'resthalf': 75, 'restinexpressible': 76, 'restquarter': 77}


In [13]:
network_input = []
network_output = []
seq_length = 20

for i in range(0, len(notes) - seq_length - 1, 1): # Take all sets of k notes as input and store the k+1 as output
    seq_in = notes[i:i + seq_length] # k notes
    seq_out = notes[i + seq_length + 1] # k+1 note
    network_input.append([note2Int[n] for n in seq_in]) # Convert to number and store input
    network_output.append(note2Int[seq_out]) # Convert to number and store output

In [14]:
network_input = np.reshape(network_input, (len(network_input), seq_length, 1))
network_input.shape
network_input = network_input / float(len(pitchList))
network_output = tf.keras.utils.to_categorical(network_output)

## Making LSTM NN Model <a class = "anchor" id = "third-section"></a> 

In [15]:
model = Sequential()
print(network_input.shape)
print(network_output.shape)

model.add(LSTM(
    256,
    input_shape = (network_input.shape[1], network_input.shape[2]), # kx1 notes on input, tells network what to expect
    return_sequences = True
))
model.add(Dropout(0.3))   #These layers can be whatever, try different combinations
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(len(pitchList))) # Last layer needs to have same size as our ouputs
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

(3811, 20, 1)
(3811, 78)


In [16]:
outputpath = "MegaManWeights_Multiple/w-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(
    outputpath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     

model.fit(network_input, network_output, epochs = 100, batch_size = 32, callbacks = callbacks_list)

Epoch 1/100
3811/3811 [==============================] - 20s 5ms/step - loss: 3.0518
Epoch 2/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.9051
Epoch 3/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.8428
Epoch 4/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.8167
Epoch 5/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.7602
Epoch 6/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.6708
Epoch 7/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.5825
Epoch 8/100
3811/3811 [==============================] - 19s 5ms/step - loss: 2.4717
Epoch 9/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.3182
Epoch 10/100
3811/3811 [==============================] - 18s 5ms/step - loss: 2.1131
Epoch 11/100
3811/3811 [==============================] - 18s 5ms/step - loss: 1.8074
Epoch 12/100
3811/3811 [==============================] - 18s 5

KeyboardInterrupt: 

## Predicting new music  <a class = "anchor" id = "fourth-section"></a>

In [17]:
predictmodel = Sequential()
predictmodel.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
predictmodel.add(Dropout(0.3))
predictmodel.add(LSTM(512, return_sequences=True))
predictmodel.add(Dropout(0.3))
predictmodel.add(LSTM(256))
predictmodel.add(Dense(256))
predictmodel.add(Dropout(0.3))
predictmodel.add(Dense(len(pitchList)))
predictmodel.add(Activation('softmax'))
predictmodel.compile(loss='categorical_crossentropy', optimizer='rmsprop')

predictmodel.load_weights('MegaManWeights_Multiple/w-16.hdf5')

In [61]:
def getPredOutput(numNotes):  
    start = np.random.randint(0, len(network_input) -1)
    pattern = network_input[start]
    pattern = [item for sublist in pattern for item in sublist]
    int2Note = dict((number, note) for number, note in enumerate(pitchList))
    

    prediction_output = []

    for note_index in range(numNotes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(len(pitchList))

        prediction = predictmodel.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int2Note[index]
        prediction_output.append(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output

In [119]:
def getOutputNotes(prediction_output, instr):   
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                if(instr == 0):
                    new_note.storedInstrument = instrument.Piano()
                else:
                    new_note.storedInstrument = instrument.Violin()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif ('rest' in pattern):
        # pattern is a rest
            new_rest = note.Rest()
            new_rest.duration.type = str(pattern[4:])
            new_rest.offset = offset
            if(instr == 0):
                new_rest.storedInstrument = instrument.Piano()
            else:
                new_rest.storedInstrument = instrument.Violin()
            output_notes.append(new_rest)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            if(instr == 0):
                new_note.storedInstrument = instrument.Piano()
            else:
                new_note.storedInstrument = instrument.Violin()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5
    return output_notes

In [120]:
predict1 = getPredOutput(100)
output1 = getOutputNotes(predict1, 0)
predict2 = getPredOutput(100)
output2 = getOutputNotes(predict2, 1)

print(output2[1].storedInstrument)

Violin


In [121]:
fp='MegaManNNOutput/MultiTest.mid'
midi_stream_piano = stream.Stream(output1)
midi_stream_violin = stream.Stream(output2)

print(midi_stream_violin)

midi_stream.write('midi', fp)

<music21.stream.Stream 0x7f337a662400>


'MegaManNNOutput/MultiTest.mid'

In [128]:
midiNN =converter.parse(fp)
for p in midiNN.parts:
     p.insert(0, instrument.Violin())
     p.insert(1, instrument.Piano())
midiNN.show('midi')
#midiNN.show()
#midiNN.show('midi')

In [90]:
for p in midiNN.parts:
    p.insert(0, instrument.Violin())

In [91]:
midiNN.show('midi')